# 項目20～22

In [31]:
import logging

## 項目20 Noneを返すのではなく例外を送出する

ユーティリティ関数を書く際、Pythonプログラマは`None`という戻り値に特別な意味を持たせる場合がある。

次の例のように、除算をするヘルパー関数を考える。ゼロで割る場合は結果が未定義となるため、`None`を返すのが関数として自然に思える。

In [32]:
# Example 1
def careful_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None

assert careful_divide(4, 2) == 2
assert careful_divide(0, 1) == 0
assert careful_divide(3, 6) == 0.5
assert careful_divide(1, 0) == None


この関数を使って、次のように戻り値が`None`でなければ戻り値を表示し、戻り値が`None`の場合は「入力が正しくない」と表示させるコードは例えば次のように書くことができる。

In [33]:
# Example 2
x, y = 1, 0
result = careful_divide(x, y)
if result is None:
    print('Invalid inputs')
else:
    print('Result is %.1f' % result)


Invalid inputs


ここで、`if`文を次のように`if not result`と書こうと試みたとする。

先ほど同様に分母がゼロのケースではExample 2と同様の挙動を取る。

しかし次のように分子がゼロで、つまり戻り値`result = 0`の場合、本来は除算の結果としてゼロを表示すべきなのに、`not result`は`True`となってしまい、分母がゼロであったときと同じ挙動を取ってしまう。

In [34]:
# Example 3
x, y = 0, 5
result = careful_divide(x, y)
if not result:
    print('Invalid inputs')  # This runs! But shouldn't
else:
    assert False

Invalid inputs


このように`None`に「分母がゼロ」という意味を持たせて`False`と同じように扱ってしまい、意図しないエラーを起こしてしまうのはPythonではよくあることである。

同様にPythonでは、次のような値がBooleanとしてFalseと見なされる：

- None
- 数値の0（0, 0.0, Decimal(0)など）
- 空のシーケンス（空の文字列、空のリスト、空のタプルなど）
- 空のマップ（空の辞書など）
- 零要素のコレクション（set()など）

上の`careful_divide`のように関数から`None`等を返すことはエラーに繋がりやすい。このエラーを減らす2つの方法を紹介する。

### 戻り値を2値のタプルにする

一つ目の方法は「戻り値を2値のタプルにする」という方法である（背景については「項目19 複数の戻り値では、4個以上の変数なら決してアンパックしない」参照）。

タプルの第1項に除算が成功したか失敗したかを`True/False`で格納し、第2項に計算結果を格納する。

In [35]:
# Example 4
def careful_divide(a, b):
    try:
        return True, a / b
    except ZeroDivisionError:
        return False, None

assert careful_divide(4, 2) == (True, 2)
assert careful_divide(0, 1) == (True, 0)
assert careful_divide(3, 6) == (True, 0.5)
assert careful_divide(1, 0) == (False, None)

この関数の呼び出し元では、戻り値のタプルをアンパックし、第1項で除算が成功したかを判定すればよい。

In [36]:
# Example 5
x, y = 5, 0
success, result = careful_divide(x, y)
if not success:
    print('Invalid inputs')

Invalid inputs


しかしこれは書き手が戻り値について調べる手間が必要である。

もし書き手がこの関数の仕様を知らず、計算結果のみを用いようと次のコードを書いてしまった場合、先ほどと同じ問題が起きてしまう。

特に、Pythonでは`_`を用いることでアンパックした要素の一部を簡単に無視できてしまうため、次のように書くことでExample 3と全く同じコードになってしまう。

In [37]:
# Example 6
x, y = 5, 0
_, result = careful_divide(x, y)
if not result:
    print('Invalid inputs')

Invalid inputs


### 特別な場合に`None`を返さない方法

上記のコードでは特別な場合（`ZeroDivisionError`の際）に`None`を返しており、それが原因で予期しないエラーが起こっていた。

そこで、`None`を返すのではなく、例外を呼び出し元に送出し、その処理を行わせるという方法を取る。

次のコードでは`ZeroDivisionError`を`ValueError`に変換し、呼び出し元に入力値が正しくないことを示している（「項目87 APIからの呼び出し元を分離するために、ルート例外を定義する」参照）。

In [38]:
# Example 7
def careful_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs')

呼び出し元では、先ほどのコードのように関数の戻り値について調べる必要はなくなった。

代わりに戻り値は常に正しいと仮定し、次のコードのように`try`の`else`ブロックで結果を出力する方法を取る（詳細は「項目65 `try`/`except`/`else`/`finally`の各ブロックを活用する」参照）。

In [39]:
# Example 8
x, y = 5, 2
try:
    result = careful_divide(x, y)
except ValueError:
    print('Invalid inputs')
else:
    print('Result is %.1f' % result)

Result is 2.5


In [40]:
# Example 8.1
x, y = 5, 0
try:
    result = careful_divide(x, y)
except ValueError:
    print('Invalid inputs')
else:
    print('Result is %.1f' % result)

Invalid inputs


さらに、次のコードのように型ヒントを用いた実装に拡張することができる。

型ヒントにより「戻り値は`float`で、`None`には絶対ならない」と指定できる（「項目90　バグを回避するために静的解析を検討する」参照）。

しかし、Pythonでは、"例外"が関数のインターフェースの一部であること（チェック例外）を示す方法を意図的に提供していない。

その代わりに、例外を送出する振る舞いをドキュメント化することにより、関数を呼び出す人はそのドキュメントを読むことで、あらかじめ例外を知るようにしておくことができる（「項目84 すべての関数、クラス、モジュールについて`docstring`を書く」参照）。

型ヒントと`docstring`を用いることで

In [41]:
# Example 9
def careful_divide(a: float, b: float) -> float:
    """Divides a by b.

    Raises:
        ValueError: When the inputs cannot be divided.
    """
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs')

try:
    result = careful_divide(1, 0)
    assert False
except ValueError:
    pass  # Expected

assert careful_divide(1, 5) == 0.2

In [42]:
# docstringはprint(関数.__doc__)で表示できる
print(careful_divide.__doc__)

Divides a by b.

    Raises:
        ValueError: When the inputs cannot be divided.
    


これにより入力・出力・例外の振る舞いが明確になり、呼び出し元が間違える回数を減らすことができる。

## 覚えておくこと

- 特別な場合に`None`等を返す関数は、条件式においてゼロや空シーケンス等と等しく`False`と評価されるため、エラーを引き起こしやすい
- `None`を返す代わりに、例外を送出して特別な条件を示すようにする。その処理を文書化しておくことで、呼び出し元のコードで適切に例外処理されることが期待できる
- 型ヒントを用いて、関数が特別な場合にも絶対に`None`を返さないことを明示できる

# 項目21 クロージャが変数スコープとどう関わるか把握しておく

## クロージャ

In [43]:
# 例1
def func():
    x = 3
    def add3(y):
        return y+x
    return add3

f = func()
print(f(4)) # 7

7


`func` 関数は `add3` 関数を定義してそれを返す。

`add3` 関数は外側で定義された `x` を関数内で使用している。

`add3` 関数をクロージャ、`func` 関数をエンクロージャという。

`x` は本来であれば `func` 数を抜けたところで消滅する`func` 関数におけるローカル変数だが、クロージャを生成したことで `x` が `add3` 関数内に記憶されて `f(4)` で呼び出した時に `x` が加算される。

## 本書部分

「数値のリストを、部の数が優先されるようにソートしたい」といった状況を考える。

1つ目の方法としては、ヘルパー関数を`key`引数として、リストの`sort`メソッドに渡すことである（詳細は「項目14 `key`引数を使い複雑な基準でソートする」参照）。

ヘルパー関数の戻り値が、リスト内の要素をソートするための値として使われる。

ヘルパーは、与えられた要素が重要なグループかどうか調べ、それに従いソートキーを変更する。

In [44]:
# Example 1
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)

関数`helper`をクロージャ、関数`sort_priority`をエンクロージャに相当する。

次のように`group`内にある数が、それ以外の数に対し優先されソートされることが確認できる。

In [45]:
# Example 2
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {3, 5, 7, 2}
sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


この関数が期待通りに働くのは、
- Pythonがクロージャをサポートしている。クロージャとは定義されたスコープの変数を参照する関数である。これにより、`helper`関数が`sort_priority`の`group`引数にアクセスできる。
- Pythonでは関数がファーストクラスオブジェクトである。つまり、直接参照でき、変数に代入したり、他の関数の引数として渡したり、式の中や`if`文の中で比較することができる。これにより、`sort`メソッドがクロージャ関数を`key`引数として受け付ける。
- Pythonでは（タプルを含めた）シーケンスの比較に特別な規則を持つ。最初にインデックス0の要素を比較し、等しければ次にインデックス1の要素を比較する。それらも等しければ、その次はインデックス2の要素となる。これにより、`helper`クロージャの戻り値がソート順で2つの異なるグループになるようにできる。

上の関数が優先度の高い要素がそもそもリストにあるか否かを返すようにできれば、ユーザーインターフェースコードがそれに応じて動けるのでより改善する。

そのような振る舞いを追加するのは簡単に思える。

それぞれの数に対してどのグループか決定するクロージャ関数は既にあるので、クロージャを用いて、高い優先度の要素があればフラグを立てれば実装できるように思える。

この方法を次のように試してみる。

In [46]:
# Example 3
def sort_priority2(numbers, group):
    found = False
    def helper(x):
        if x in group:
            found = True  # Seems simple
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

In [47]:
# Example 4
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
found = sort_priority2(numbers, group)
print('Found:', found)
print(numbers)

Found: False
[2, 3, 5, 7, 1, 4, 6, 8]


ソートした結果は正しいが、`group`の要素があるのに`found`は`False`となってしまっている。

式の中の変数を参照するとき、Pythonインタプリタはスコープ内を横断して、次の順序で参照を解決する。

- 現在の関数のスコープ
- （他の関数の中にある場合のように）外側のスコープ
- コードを含むモジュールのスコープ（グローバルスコープとも呼ばれる）
- （`len`や`str`のような関数を含む）組み込みスコープ

これらのいずれにも参照名の定義済み変数がないと、`NameError`例外を送出する。

In [48]:
# Example 5
try:
    foo = does_not_exist * 5
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_11432\3528649612.py", line 3, in <cell line: 2>
    foo = does_not_exist * 5
NameError: name 'does_not_exist' is not defined


変数への値代入は、これとは動きが異なる。

①変数が現在のスコープですでに定義されていると、その変数は新たな値をとる

②変数が現在のスコープに存在しないと、Pythonは代入を変数定義のように扱う。新たに定義された変数のスコープは代入を含む関数である。

この代入での振る舞いが、`sort_priority2`関数の戻り値がおかしくなってしまう原因である。

変数`found`は`helper`クロージャの中で`True`と代入されている。

この時"`helper`クロージャでの代入"は、`helper`内の新たな変数定義として扱われ（②）、`sort_priority2`内の代入（①）としては扱われていなかったのである。

In [49]:
# Example 6
def sort_priority2(numbers, group):
    found = False         # Scope: 'sort_priority2'
    def helper(x):
        if x in group:
            found = True  # Scope: 'helper' -- Bad!
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

この手の問題は、Pythonの初学者を混乱させるため、「スコープ処理バグ」と呼ばれることがある。

しかし、これはバグではなく、仕様上の意図された結果である。

逆に、この振る舞いのおかげで、関数のローカル変数はそれを含んだモジュールを汚染せずに済むのである。

そうでなければ、関数内のすべての代入がグローバルモジュールスコープと同義となり、むしろバグの原因となりうるだろう。

### `nonlocal`構文

Pythonでは、データをクロージャの外に出す特別な構文がある。

`nonlocal`文が、指定した変数名の代入に際してスコープ横断をすべきことを示す。

唯一の制限としては、`nonlocal`が（グローバルを汚染しないように）モジュールレベルのスコープまでは行かないことである。

`nonlocal`を用いて次のように同じ関数を定義する。

In [50]:
# Example 7
def sort_priority3(numbers, group):
    found = False
    def helper(x):
        nonlocal found  # Added
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

In [51]:
# Example 8
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
found = sort_priority3(numbers, group)
assert found
assert numbers == [2, 3, 5, 7, 1, 4, 6, 8]

データが代入されるときに、それがクロージャの外のスコープにあることを`nonlocal`文によって明示する。

これは、変数の代入が直接モジュールのスコープになる`global`文と補完関係にある。

しかし、グローバル変数のアンチパターンと同様に、`nonlocal`を単純な関数以外には使わないほうが好ましい。

`nonlocal`の抱える副作用は見つけることが困難だからである。

大きな関数で、`nonlocal`文と関連する変数への代入が離れていると、特に理解が困難になる。

### ヘルパー関数でラップ

`nonlocal`の使い方が複雑になってきたら、状態をヘルパークラスでラップするとよい。

次のように、`nonlocal`を使う方式と同じ結果を出すクラスを定義する。

少し長くなっているが、読みやすくなっている（特殊メソッド`__call__`の詳細は「項目38 単純なインターフェースにはクラスの代わりに関数を使う」参照）。

In [52]:
# Example 9
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {3, 5, 7, 2}
class Sorter:
    def __init__(self, group):
        self.group = group
        self.found = False

    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)

sorter = Sorter(group)
numbers.sort(key=sorter)
assert sorter.found is True # Trueであることが確認できる
assert numbers == [2, 3, 5, 7, 1, 4, 6, 8]

## 覚えておくこと

- クロージャ関数は定義されたスコープのどれからでも変数を参照できる
- デフォルトではクロージャ内での変数への代入は外部のスコープには影響しない
- `nonlocal`文を用いてクロージャが外のスコープにある変数を修正できる
- `nonlocal`文を単純な関数でのみ使うようにする

# 項目22 可変長位置引数を使って、見た目をすっきりさせる

可変長位置引数（`varargs`または`*args`という記法から「スター引数(star args)」とも呼ぶ）を使うと、関数呼び出しがより明確で見た目がすっきりする。

例えば、デバッグ情報のログを取ることを考える。

固定個数の引数だと、次のようにメッセージと値のリストを取る関数が必要である。

In [53]:
# Example 1
def log(message, values):
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{message}: {values_str}')

log('My numbers are', [1, 2])
log('Hi there', [])

My numbers are: 1, 2
Hi there


値がないときに第二引数に空のリストを`log`に渡さないといけないのが面倒ですっきりしていない。

Pythonでは、最後の位置引数の名前に`*`をつけることで、このような渡したくない空の要素を省くように実装することができる。

今実装ではログ用メッセージの第一引数は不可欠だが、次の位置引数はオプションで何個でも構わない。

次のように関数本体は全く同じで、引数の部分だけ変えて実装できる。

In [54]:
# Example 2
def log(message, *values):  # The only difference
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{message}: {values_str}')

log('My numbers are', 1, 2)
log('Hi there')  # Much better

My numbers are: 1, 2
Hi there


この構文は「項目13　スライスではなくcatch-allアンパックを使う」で出てきたアンパック代入文で使うアスタリスク付きの式によく似ている。

（`list`などの）シーケンスで`log`のような可変個引数関数を呼び出したいなら、*演算子を使って呼び出すことができる。

Pythonにシーケンスの要素を位置引数に渡すよう指示すればよい。

In [55]:
# Example 3
favorites = [7, 33, 99]
log('Favorite colors', *favorites)

Favorite colors: 7, 33, 99


※ 上の`log`では実際には*ではなくリストそのものを渡しても実行結果は変わらない

※ 下の例のようにそのものを取り出してみると挙動の違いに気づく。そのまま`list`で渡すと期待した挙動とならないが、`*list`の形で渡すことで各要素を要素に持つタプルの形式で渡す挙動を取る（後述）

In [56]:
log('Favorite colors', favorites)

Favorite colors: [7, 33, 99]


In [57]:
def test(*args):
    print(args)

test([1,2])
test(*[1,2])

([1, 2],)
(1, 2)


可変長位置引数を使うには2つの問題が生じる。

### 問題点1：可変個数の引数が、関数に渡される前に常にタプルに変換される

問題点の一つ目は可変個数の引数が、関数に渡される前に常にタプルに変換されているということである。

つまり関数の呼び出し元がジェネレータで*演算子を使っていれば、終わるまでイテレーションされてしまう（「項目30 リストを返さずにジェネレータを返すことを考える」参照）。

結果のタプルにはジェネレータからのすべての値が含まれるため、メモリを大量に消費して、プログラムをクラッシュさせる危険性がある。

In [58]:
# Example 4
def my_generator():
    for i in range(10):
        yield i

def my_func(*args):
    print(args)

it = my_generator()
my_func(*it)

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)


`*args`を受け入れる関数は、引数リストの入力個数が少ないことがわかっている場合が一番適している。

多くのリテラルや変数名を一緒に渡す関数呼び出しが理想的である。

基本的にプログラマの便宜のためであり、コードの可読性が高くなる。

### 問題点2：新たな位置パラメータを追加する際に発見困難なバグを生じうる

`*args`の第二の問題点は、すべての呼び出し元を修正せずには関数に対して新たな位置引数を追加できないことである。

引数リストの前に位置引数を追加すれば、既存の呼び出し元は、更新されない限り気づかない内に壊れてしまう。

In [59]:
# Example 5
def log(sequence, message, *values):
    if not values:
        print(f'{sequence} - {message}')
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{sequence} - {message}: {values_str}')

log(1, 'Favorites', 7, 33)      # New with *args OK
log(1, 'Hi there')              # New message only OK
log('Favorite numbers', 7, 33)  # Old usage breaks

1 - Favorites: 7, 33
1 - Hi there
Favorite numbers - 7: 33


ここでの問題は、`log`の3番目の呼び出しで、`sequence`引数がないので7を`message`パラメータとしてしまうことである。

このようなバグは、コードが何の例外も起こさず実行をしてしまうため、探し出すのが困難だという点である。

このような危険性を完全になくすには、`*args`を受け入れる関数を拡張するときには、キーワード専用引数を使う（「項目25 キーワード専用引数と位置専用引数で明確さを高める」参照）ことである。

より安全にするには、型ヒントを使うこと（「項目90 バグを回避するために静的解析を検討する」参照）も検討すべきである。

## 覚えておくこと
- 関数は`def`文で`*args`を使うことにより、可変個数の位置引数を受け入れられる
- `*`演算子を関数に用いて、シーケンスからの要素を位置引数として使える
- `*`演算子をジェネレータと一緒に使うと、プログラムがメモリを使い果たしてクラッシュすることがある
- `*args`を使う関数に新たに位置パラメータを追加すると、発見が困難なバグを生み出す危険性がある